# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [6]:
# imports
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import ollama

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

OLLAMA_API = "http://localhost:11434/api/chat"
# either use this key with requests or simply use the ollama package


In [4]:
# here is the question; type over this to ask something new

system_prompt = "You are a great joyful assistant"

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [9]:
# Get gpt-4o-mini to answer, with streaming

response = openai.chat.completions.create(model=MODEL_GPT, 
                                          messages=[{"role":"system", "content": system_prompt},
                                                    {"role":"user", "content": question}])

# additional available parameter : 
    # response_format={"type": "json_object"}, for a dictionnary result
        # which then can be handled like this :
        #result = json.loads(response.choices[0].message.content)

#print(response.choices[0].message.content)
display(Markdown(response.choices[0].message.content))

This code snippet uses Python's `yield from` statement along with a set comprehension to iterate over a collection of books and yield the authors' names.

Here’s a breakdown of the components:

1. **Set Comprehension**: 
   - `{book.get("author") for book in books if book.get("author")}` generates a set of unique authors from the `books` collection.
   - `book.get("author")` attempts to retrieve the value associated with the key `"author"` from each `book` dictionary.
   - The `if book.get("author")` filters out any books that do not have an author, ensuring that only books with a valid author value are included in the final set.

2. **`yield from`**: 
   - The `yield from` statement is used to yield all values from the iterator returned by the expression on its right. In this case, it yields each unique author found in the set produced by the set comprehension.
   - `yield from` simplifies the generator function by yielding each value from the iterable, rather than looping through it and yielding each one individually.

### Why Use This Code?
- **Efficiency**: Since a set is used, this code produces a collection of unique authors, eliminating duplicates across the books.
- **Readability**: The use of set comprehension provides a concise way of gathering authors, making the code easier to read and understand.
- **Generator Functionality**: Using `yield from` allows the function to act as a generator, enabling iteration over authors when called, which is more memory-efficient than collecting all authors in a list before returning them.

### Example:
If `books` is defined as follows:
```python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": "Author A"},
    {"title": "Book Four"}  # No author
]
```

Running the provided code would yield:
```
'Author A'
'Author B'
```

These are the unique authors, with duplicates removed and ignoring the `book` without an author.

---

To make it streaming, day5 explains you can add `stream = True` in `stream = openai.chat.completions.create()` and then 
```python
response = ""
    display_handle = display(Markdown(""), 
                             display_id=True)
    for chunk in stream: #stream is the name of the completions.create result
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
```

In [11]:
# Get Llama 3.2 to answer

HEADERS = {"Content-Type": "application/json"}
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]
payload = {
        "model": MODEL_LLAMA,
        "messages": messages,
        "stream": True
    }

# OLLAMA requests
#response = requests.post(OLLAMA_API, json=payload, headers=HEADERS).json()

# OLLAMA package
response = ollama.chat(model=MODEL_LLAMA, messages=messages)

display(Markdown(response['message']['content']))

**Breaking Down the Code**

This line of code is written in Python and utilizes a feature called **generator expressions**, which allows us to create an iterable directly within the syntax of the `for` loop.

Here's what each part does:

- `{book.get("author") for book in books if book.get("author")}`: This is a generator expression that:
  - Iterates over a collection (in this case, `books`) using the `for book in books` syntax.
  - Filters out any items in the collection for which the key `"author"` does not exist or its value is empty (`book.get("author") == None` or `book.get("author") == ""`). This is done using the `if book.get("author")` condition.
  - For each valid item, it extracts and returns the corresponding value.

- `yield from {...}`: This keyword is used to delegate the iteration to another iterable (in this case, the generator expression above). It tells Python to yield each result one by one, instead of building an entire list or array in memory.

**What Does It Do?**

When you put it all together, this line of code will create a new iterator that yields only the values corresponding to the `"author"` key for each item in the `books` collection where the `"author"` key exists. The resulting iterable is then delegated to Python's built-in iteration mechanism.

In essence, it performs a **lazy filtering** operation on the `books` collection, which can be beneficial when working with large datasets or memory-constrained environments.

**Example Use Case**

Here's an example of how this might be used in practice:

```python
# Assuming books is a list of dictionaries, where each dictionary represents a book.
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author C"}
]

# Use the generator expression
for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)  # Output: Author A, Author C

# Compare to using a list comprehension and iterating over it
authors = [book["author"] for book in books if "author" in book]
for author in authors:
    print(author)  # Output: Author A, Author C
```

In the first example, we use `yield from` to create an iterator that produces only the authors of the books with existing `"author"` keys. In contrast, the second example uses a list comprehension to filter and store all valid authors in memory before iterating over them. The former approach is more memory-efficient for large datasets.

---

day2_exercize shows yet another possibility : using the OpenAI client python library to call Ollama.


In [ ]:

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=messages
)

print(response.choices[0].message.content)